# Tarea algoritmos de optimización con gradiente
autor: Emmanuel A. Larralde Ortiz

In [ ]:
from copy import copy
from sympy import(
    Matrix,
    symbols,
    latex,
    symbols,
    Symbol,
    init_printing
)
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def Grad(f, variables):
    return Matrix([f]).jacobian(Matrix(symbols(variables))).T

In [ ]:
def rosenbrock(x):
  x1 = x[0]
  x2 = x[1]
  x3 = x[2]
  x4 = x[3]
  return 100*(x1**2-x2)**2 + (x1-1)**2 + 100*(x3**2-x4)**2 + (x3-1)**2

# Problema
Encontrar el punto óptimo de la siguiente **ecuación**

$$\sum _{i=1}^{2} [100(x_{2i-1}^2-x_{2i})^2 + (x_{2i-1}-1)^2]$$

Desarrollando:

$$100(x_1^2 - x_2)^2 + (x_1 - 1)^2 + 100(x_3^2 - x_4)^2 + (x_3 - 1)^2$$

A través de cómputo simbólico obtengamos las ecuaciones de gradiente


In [ ]:
x = [Symbol(f'x{i+1}') for i in range(10)] #x[0] = x_1, x[1] = x_1, ...

Entonces, la ecuación se define de la siguiente forma


In [ ]:
f = 100*(x[0]**2 - x[1])**2 + (x[0] - 1)**2 + 100*(x[2]**2 - x[3])**2 + (x[2] - 1)**2 
f

Gradiente de la función


In [ ]:
Grad(f, 'x1, x2, x3, x4')

Numéricamente

In [ ]:
def grad(x):
  x1 = x[0]
  x2 = x[1]
  x3 = x[2]
  x4 = x[3]

  gradient = np.array([
   400*x1*(x1**2 - x2) + 2*x1 -2,
   -200*x1**2 + 200*x2,
   400*x3*(x3**2 - x4) + 2*x3 - 2,
   -200*x3**2 + 200*x4
  ])
  return gradient

## Algoritmos de optimización con gradiente

Descenso del gradiente

In [ ]:
#x_init = 0.5*np.random.normal(size=4)
x_init = np.array([0.0, 0.0, 0.0, 0.0])

In [ ]:
def GD(grad=None, gd_params={}, f_params={}):
    """
    Gradient Descent
    """
    nIter = gd_params['nIter']
    alpha = gd_params['alpha']
    theta = copy(f_params['X'])
    Theta = []
    for _ in range(nIter):
        p = grad(theta)
        theta -= alpha*p
        Theta.append(copy(theta))
    return np.array(Theta)

In [ ]:
#x_init = 0.5*np.random.normal(size=4)
gd_params = {
    'alpha' : 0.001, 
    'nIter' : 10000,
}
f_params={'X': x_init}

In [ ]:
sol_gd = GD(grad=grad, gd_params=gd_params, f_params=f_params)

In [ ]:
def NAG(grad=None, gd_params={}, f_params={}):
    """
    Nesterov Accelerated Gradient
    """
    nIter = gd_params['nIter']
    alpha = gd_params['alpha'] 
    eta   = gd_params['eta']
    theta = copy(f_params['X'])
    p = np.zeros(theta.shape)
    Theta = []
    for _ in range(nIter):
        pre_theta = theta - 1.0*alpha*p
        g = grad(pre_theta)
        p = g + eta*p
        theta = theta - alpha*p
        Theta.append(copy(theta))
    return np.array(Theta)

In [ ]:
#x_init = 0.5*np.random.normal(size=4)
# parámetros de los algoritmos
gd_params = {
    'alpha'          : 0.001,
    'nIter'          : 10000,
    'eta'            : 0.9,
}

# parámetros de la función objetivo
f_params={'X': x_init}

In [ ]:
sol_nag = NAG(grad=grad, gd_params=gd_params, f_params=f_params)

Descenso de Gradiente Adaptable con Momentum(A DAM) 

In [ ]:
def ADAM(grad=None, gd_params={}, f_params={}):
    '''
    Descenso de Gradiente Adaptable con Momentum(A DAM) 
    '''
    epsilon= 1e-8
    nIter    = gd_params['nIter']
    alpha    = gd_params['alphaAdam'] 
    eta1     = gd_params['eta1']
    eta2     = gd_params['eta2']
    theta    = f_params['X']
    p        = np.zeros(theta.shape)
    v        = 0.0
    Theta    = []
    eta1_t = eta1
    eta2_t = eta2
    for t in range(nIter):
        g  = grad(theta)
        p  = eta1*p + (1.0-eta1)*g
        v  = eta2*v + (1.0-eta2)*(g**2)
        ph = p/(1.0-eta1_t)
        vh = v/(1.0-eta2_t)
        theta = theta - alpha * ph / (np.sqrt(vh)+epsilon)
        eta1_t *= eta1
        eta2_t *= eta2
        Theta.append(copy(theta))
    return np.array(Theta)

In [ ]:
#x_init = 0.5*np.random.normal(size=4)
# parámetros de los algoritmos
gd_params = {
    'alphaAdam'      : 0.01,
    'nIter'          : 10000,
    'eta'            : 0.9,
    'eta1'           : 0.9,
    'eta2'           : 0.999
}

# parámetros de la función objetivo
f_params={'X': x_init}

In [ ]:
sol_adam = ADAM(grad=grad, gd_params=gd_params, f_params=f_params)
sol_adam[-1]

In [ ]:
def array_rosenbrock(x):
    x1 = np.array([e[0] for e in x])
    x2 = np.array([e[1] for e in x])
    x3 = np.array([e[2] for e in x])
    x4 = np.array([e[3] for e in x])
    return 100*(x1**2-x2)**2 + (x1-1)**2 + 100*(x3**2-x4)**2 + (x3-1)**2

In [ ]:
plt.plot(array_rosenbrock(sol_gd[:5000]), label='Simple GD')
plt.plot(array_rosenbrock(sol_nag[:5000]), label='NAG')
plt.plot(array_rosenbrock(sol_adam[:5000]), label='ADAM')
plt.legend()

In [ ]:
plt.plot(array_rosenbrock(sol_adam[:1000]), label='ADAM')
plt.plot(array_rosenbrock(sol_nag[:1000]), label='NAG')